# Libraries

In [1]:
import os
import xarray as xr
import rioxarray as rxr

In [2]:
main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\FAPAR anomaly\\'
# Directory for monthly FAPAR ANomaly
FAPAR_anomaly = os.path.join(main_dir, 'FAPAR Anomaly','RT1')
# Directory for crop mask for each month
Crop_mask = os.path.join(main_dir,'Crop_mask')

In [4]:
# Dictionary includes crop mask of each month
# with the key as the name of the month
Crop_mask_month ={}
for image in os.listdir(Crop_mask):
    path = os.path.join(Crop_mask, image)
    crop_mask = rxr.open_rasterio(path, masked=True)
    Crop_mask_month[image[:-4]]=crop_mask 

In [6]:
# Looping through each monthly FAPAR anomaly and mask values based on 
# monthly crop mask
for image in os.listdir(FAPAR_anomaly):
    # path for monthly FAPAR anomaly
    path = os.path.join(FAPAR_anomaly, image)
    # open monthly FAPAR anomaly
    FAPAR_a = rxr.open_rasterio(path, masked=True)
    # Call the corresponding monthly crop mask for the same month as FAPAR anomaly
    # by taking the month from the name of monthly FAPAR anomaly
    monthly_crop_mask = Crop_mask_month[image[:-22]]
    # Return VCI values which only match the crop mask, and return -9999 if not
    FAPAR_anomaly_mask = xr.where(monthly_crop_mask ==1, FAPAR_a[0], -9999)
    # Convert -9999 values to NAN
    FAPAR_anomaly_mask = FAPAR_anomaly_mask.where(FAPAR_anomaly_mask!=-9999)
    # CRS
    FAPAR_anomaly_mask= FAPAR_anomaly_mask.rio.write_crs(FAPAR_a.rio.crs)
    # Export VCI after crop masking
    FAPAR_anomaly_mask.rio.to_raster(main_dir + '\\' +'FAPAR Anomaly_crop_mask'+ '\\'+ 'RT1'+'\\'+image[:-4]+'_crop_mask.tif')
    